In [29]:
import plotly.express as px
import pandas as pd
import dash_leaflet as dl
import geopandas
import dash_leaflet.express as dlx
from xgboost import XGBRegressor
import pickle
import json

In [4]:
df_wmoid = pd.read_excel('./Data/daftar_wmoid.xlsx')
df_wmoid = df_wmoid.rename(columns={'WMOID': 'lokasi'}) # WMOID dataframe preproceses
df_wmoid = df_wmoid[['lokasi', 'Nama UPT']]
df_wmoid

,lokasi,Nama UPT
0,96001,Stasiun Meteorologi Maimun Saleh
1,96009,Stasiun Meteorologi Malikussaleh
2,96011,Stasiun Meteorologi Sultan Iskandar Muda
3,96013,Stasiun Geofisika Aceh Besar
4,96015,Stasiun Meteorologi Cut Nyak Dhien Nagan Raya
...,...,...
180,97876,Stasiun Meteorologi Tanah Merah
181,97900,Stasiun Meteorologi Mathilda Batlayeri
182,97978,Stasiun Klimatologi Merauke
183,97980,Stasiun Meteorologi Mopah


In [5]:
ina_nwp_input = pd.read_csv('./Data/MONAS-input_nwp_compile.csv') # Feature to predict from INA-NWP
ina_nwp_input

,lokasi,Date,suhu2m(degC),dew2m(degC),rh2m(%),wspeed(m/s),wdir(deg),lcloud(%),mcloud(%),hcloud(%),...,ws800(m/s),wd800(deg),t500(degC),rh500(%),ws500(m/s),wd500(deg),prec_nwp,LAT,LON,ELEV
0,96001,2023-10-17 15:00:00,27.4007,23.6856,80.3053,4.25554,191.08600,0.374474,1.000000,0.855219,...,8.18965,242.5890,-4.71925,94.9932,4.68350,76.8328,2.923480e-19,5.87655,95.33785,126
1,96001,2023-10-17 18:00:00,27.4089,23.0971,77.4673,3.20000,228.61600,0.262561,1.000000,0.872848,...,6.67126,204.7110,-4.99763,95.7026,4.83021,124.4510,1.034630e-19,5.87655,95.33785,126
2,96001,2023-10-17 21:00:00,27.3820,23.8747,81.3127,6.43993,224.35900,0.431500,1.000000,0.849396,...,5.34901,203.9090,-5.10086,95.3166,4.00113,107.2340,0.000000e+00,5.87655,95.33785,126
3,96001,2023-10-18 00:00:00,27.4679,23.7839,80.4639,6.91585,220.91600,0.317385,1.000000,0.524404,...,4.76448,191.2090,-4.39951,84.1607,3.31394,114.2420,0.000000e+00,5.87655,95.33785,126
4,96001,2023-10-18 03:00:00,27.9673,23.2775,75.7992,6.28717,214.11300,0.252877,0.785813,0.610115,...,3.94272,217.0360,-4.74009,86.5560,3.79705,166.5850,0.000000e+00,5.87655,95.33785,126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4147,99992,2023-10-20 00:00:00,26.8513,19.0878,62.5816,1.52901,146.67200,0.051358,0.210951,0.000000,...,9.53871,94.4738,-4.78717,37.3070,3.18159,304.7020,0.000000e+00,-6.30253,106.75636,41
4148,99992,2023-10-20 03:00:00,32.5296,17.3364,40.4115,2.83196,76.91420,0.000000,0.227088,0.000000,...,10.88190,99.9526,-4.34492,31.4119,2.67764,274.8970,0.000000e+00,-6.30253,106.75636,41
4149,99992,2023-10-20 06:00:00,33.2196,21.2222,49.5108,6.39935,10.12190,0.000000,0.495652,0.000000,...,6.33450,121.2080,-4.53275,34.5896,2.30676,282.9150,3.250240e-03,-6.30253,106.75636,41
4150,99992,2023-10-20 09:00:00,31.2358,21.8844,57.6837,5.41735,1.79568,0.000000,0.589733,0.000000,...,7.09150,106.4220,-4.10275,29.5415,2.16724,244.8240,1.239780e-05,-6.30253,106.75636,41


In [7]:
# Make merged dataframe for mapping
df_map = df_wmoid.merge(ina_nwp_input, on='lokasi')
df_map = df_map[['lokasi', 'Nama UPT', 'LON', 'LAT']].drop_duplicates()
df_map

,lokasi,Nama UPT,LON,LAT
0,96001,Stasiun Meteorologi Maimun Saleh,95.33785,5.87655
24,96009,Stasiun Meteorologi Malikussaleh,96.94749,5.22869
48,96011,Stasiun Meteorologi Sultan Iskandar Muda,95.41700,5.52244
72,96015,Stasiun Meteorologi Cut Nyak Dhien Nagan Raya,96.24796,4.04928
96,96017,Stasiun Klimatologi Aceh,95.46400,5.40400
...,...,...,...,...
3936,97876,Stasiun Meteorologi Tanah Merah,140.31000,-6.10000
3960,97900,Stasiun Meteorologi Mathilda Batlayeri,131.30000,-7.98000
3984,97978,Stasiun Klimatologi Merauke,140.51700,-8.38700
4008,97980,Stasiun Meteorologi Mopah,140.41568,-8.52019


In [9]:
# INA-NWP input preprocess
ina_nwp_input_filtered = ina_nwp_input.drop(columns=['Date', 'LAT', 'LON'])  
ina_nwp_input_filtered = ina_nwp_input_filtered.rename(
    columns={
        'suhu2m(degC)' : 'suhu2m.degC.',
        'dew2m(degC)' : 'dew2m.degC.',
        'rh2m(%)' : 'rh2m...',
        'wspeed(m/s)' : 'wspeed.m.s.',
        'wdir(deg)' : 'wdir.deg.',
        'lcloud(%)' : 'lcloud...',
        'mcloud(%)' : 'mcloud...' ,
        'hcloud(%)' : 'hcloud...',
        'surpre(Pa)' : 'surpre.Pa.' ,
        'clmix(kg/kg)' : 'clmix.kg.kg.' ,
        'wamix(kg/kg)' : 'wamix.kg.kg.' ,
        'outlr(W/m2)' : 'outlr.W.m2.' ,
        'pblh(m)' : 'pblh.m.',
        'lifcl(m)' : 'lifcl.m.' ,
        'cape(j/kg)' : 'cape.j.kg.' ,
        'mdbz' : 'mdbz' ,
        't950(degC)' : 't950.degC.' ,
        'rh950(%)' : 'rh950...',
        'ws950(m/s)' : 'ws950.m.s.' ,
        'wd950(deg)' : 'wd950.deg.' ,
        't800(degC)' : 't800.degC.' ,
        'rh800(%)' : 'rh800...' ,
        'ws800(m/s)' : 'ws800.m.s.',
        'wd800(deg)' : 'wd800.deg.' ,
        't500(degC)' : 't500.degC.' ,
        'rh500(%)' : 'rh500...' ,
        'ws500(m/s)' : 'ws500.m.s.' ,
        'wd500(deg)' : 'wd500.deg.',
})
ina_nwp_input_filtered

,lokasi,suhu2m.degC.,dew2m.degC.,rh2m...,wspeed.m.s.,wdir.deg.,lcloud...,mcloud...,hcloud...,surpre.Pa.,...,t800.degC.,rh800...,ws800.m.s.,wd800.deg.,t500.degC.,rh500...,ws500.m.s.,wd500.deg.,prec_nwp,ELEV
0,96001,27.4007,23.6856,80.3053,4.25554,191.08600,0.374474,1.000000,0.855219,100903.0,...,16.0404,73.3359,8.18965,242.5890,-4.71925,94.9932,4.68350,76.8328,2.923480e-19,126
1,96001,27.4089,23.0971,77.4673,3.20000,228.61600,0.262561,1.000000,0.872848,100735.0,...,15.9013,73.9679,6.67126,204.7110,-4.99763,95.7026,4.83021,124.4510,1.034630e-19,126
2,96001,27.3820,23.8747,81.3127,6.43993,224.35900,0.431500,1.000000,0.849396,100655.0,...,15.6937,73.0260,5.34901,203.9090,-5.10086,95.3166,4.00113,107.2340,0.000000e+00,126
3,96001,27.4679,23.7839,80.4639,6.91585,220.91600,0.317385,1.000000,0.524404,100769.0,...,15.4617,74.9905,4.76448,191.2090,-4.39951,84.1607,3.31394,114.2420,0.000000e+00,126
4,96001,27.9673,23.2775,75.7992,6.28717,214.11300,0.252877,0.785813,0.610115,100950.0,...,14.9336,76.5370,3.94272,217.0360,-4.74009,86.5560,3.79705,166.5850,0.000000e+00,126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4147,99992,26.8513,19.0878,62.5816,1.52901,146.67200,0.051358,0.210951,0.000000,100571.0,...,15.3491,71.3439,9.53871,94.4738,-4.78717,37.3070,3.18159,304.7020,0.000000e+00,41
4148,99992,32.5296,17.3364,40.4115,2.83196,76.91420,0.000000,0.227088,0.000000,100550.0,...,15.5818,60.1363,10.88190,99.9526,-4.34492,31.4119,2.67764,274.8970,0.000000e+00,41
4149,99992,33.2196,21.2222,49.5108,6.39935,10.12190,0.000000,0.495652,0.000000,100316.0,...,15.9853,67.9167,6.33450,121.2080,-4.53275,34.5896,2.30676,282.9150,3.250240e-03,41
4150,99992,31.2358,21.8844,57.6837,5.41735,1.79568,0.000000,0.589733,0.000000,100266.0,...,15.8179,74.5408,7.09150,106.4220,-4.10275,29.5415,2.16724,244.8240,1.239780e-05,41


In [11]:
# Functions
def predict_xgb(model_path, columns_to_drop, input_df):
    model = XGBRegressor()
    model.load_model(model_path)
    input_data = input_df.drop(columns=columns_to_drop)
    predictions = model.predict(input_data)
    return predictions

def create_data_table(df_wmoid, df_pred, col_name, merge_column='lokasi'):
    df_pred['Date'] = pd.to_datetime(df_pred['Date'])
    df_pred['day'] = df_pred['Date'].dt.day

    # Group by 'day' and calculate the averages
    df_pred_avg = df_pred.groupby(['lokasi', 'day']).agg({
        'prediction': ['max', 'mean', 'min']
    }).astype('float64').round(1).reset_index()

    # Rename columns
    df_pred_avg.columns = ['lokasi', 'day', f'max_{col_name}', f'mean_{col_name}', f'min_{col_name}']

    # Convert 'Date' to a string format without timezone
    df_pred_avg['Date'] = df_pred_avg['day'].apply(lambda x: f'2023-10-{x:02d}')

    # Rearrange columns
    df_pred_avg = df_pred_avg[['lokasi', 'day', 'Date', f'max_{col_name}', f'mean_{col_name}', f'min_{col_name}']]

    # Merge with df_wmoid
    data_table_lokasi = pd.merge(df_wmoid, df_pred_avg, how='inner', on=merge_column)

    return data_table_lokasi

def plot_graph(df_graph, upt_name, nwp_output, graph_type):
    if graph_type in ['Temperature', 'Humidity'] : 
        figure = px.line(
                df_graph, 
                x='Date', 
                y='prediction', 
                title=f'{graph_type} in UPT {upt_name} (UTF)', 
                markers=True, 
                line_shape='spline'
                )
        
        figure.add_scatter(
                x=df_graph['Date'], 
                y=df_graph[nwp_output], 
                mode='lines', 
                name=f'Output {graph_type} 2m INA-NWP',
                line_shape='spline'
                )
        
        figure.update_layout(
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1,
                )
            )
        
        return figure
    elif graph_type in ['Precipitation']:
        figure = px.bar(
                df_graph, 
                x='Date', 
                y=['prediction', 'prec_nwp'], 
                title=f'{graph_type} in UPT {upt_name} (UTF)', 
                )
        return figure
    
def generate_output_df(variable_name, original_df, filtered_df, prediction_array):
    df_pred = pd.concat([original_df['Date'], filtered_df[['lokasi', variable_name]], pd.Series(prediction_array, index=filtered_df.index)], axis=1)
    df_pred.columns = ['Date', 'lokasi', variable_name, 'prediction']
    df_pred = df_pred.dropna()
    return df_pred

In [13]:
# Load ML Models
with open('./models/huber_regressor_bad.pkl','rb') as f:
    prec_model = pickle.load(f)

# Predict precipitation
prec_pred = prec_model.predict(ina_nwp_input_filtered[[
    'lokasi', 'suhu2m.degC.', 'dew2m.degC.', 'rh2m...', 'wspeed.m.s.',
    'wdir.deg.', 'lcloud...', 'mcloud...', 'hcloud...', 'surpre.Pa.',
    'clmix.kg.kg.', 'wamix.kg.kg.', 'outlr.W.m2.', 'pblh.m.', 'lifcl.m.',
    'cape.j.kg.', 'mdbz', 't950.degC.', 'rh950...', 'ws950.m.s.',
    'wd950.deg.', 't800.degC.', 'rh800...', 'ws800.m.s.', 'wd800.deg.',
    't500.degC.', 'rh500...', 'ws500.m.s.', 'wd500.deg.', 'ELEV',
    'prec_nwp'
]])
prec_pred

array([ 0.31072926,  0.28231343,  0.24538189, ..., -0.137372  ,
       -0.03611404,  0.23338712])

In [14]:
# Predict temperature
temp_model_path = './models/Temp_xgb_tuned_RMSE_1_441.json'
temp_columns_to_drop = ['lcloud...', 'mcloud...', 'hcloud...', 'clmix.kg.kg.', 'wamix.kg.kg.', 'prec_nwp']
temp_pred = predict_xgb(temp_model_path, temp_columns_to_drop, ina_nwp_input_filtered)
temp_pred

c:\Users\nasut\anaconda3\envs\Oppenheimer\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\nasut\anaconda3\envs\Oppenheimer\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\nasut\anaconda3\envs\Oppenheimer\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\nasut\anaconda3\envs\Oppenheimer\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  r

array([26.325342, 26.138168, 26.610733, ..., 33.51495 , 32.544765,
       29.047167], dtype=float32)

In [15]:
# Predict humidity
humid_model_path = './models/xgbregressor_humidity.json'
humid_columns_to_drop = ['prec_nwp', 'mcloud...', 'wamix.kg.kg.', 'clmix.kg.kg.', 'lcloud...', 'hcloud...']
humid_pred = predict_xgb(humid_model_path, humid_columns_to_drop, ina_nwp_input_filtered)
humid_pred

c:\Users\nasut\anaconda3\envs\Oppenheimer\lib\site-packages\xgboost\core.py:160: UserWarning: [19:09:27] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\learner.cc:873: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
  warnings.warn(smsg, UserWarning)
c:\Users\nasut\anaconda3\envs\Oppenheimer\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\nasut\anaconda3\envs\Oppenheimer\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\nasut\anaconda3\envs\O

array([83.37135 , 84.75032 , 81.520134, ..., 48.920624, 56.70622 ,
       66.43083 ], dtype=float32)

In [17]:
# OUTPUT temperature
df_pred_temp = generate_output_df('suhu2m.degC.', ina_nwp_input, ina_nwp_input_filtered, temp_pred)
df_pred_temp

,Date,lokasi,suhu2m.degC.,prediction
0,2023-10-17 15:00:00,96001,27.4007,26.325342
1,2023-10-17 18:00:00,96001,27.4089,26.138168
2,2023-10-17 21:00:00,96001,27.3820,26.610733
3,2023-10-18 00:00:00,96001,27.4679,26.819075
4,2023-10-18 03:00:00,96001,27.9673,27.512775
...,...,...,...,...
4147,2023-10-20 00:00:00,99992,26.8513,26.116001
4148,2023-10-20 03:00:00,99992,32.5296,32.549381
4149,2023-10-20 06:00:00,99992,33.2196,33.514950
4150,2023-10-20 09:00:00,99992,31.2358,32.544765


In [18]:
# OUTPUT humidity
df_pred_humid = generate_output_df('rh2m...', ina_nwp_input, ina_nwp_input_filtered, humid_pred)
df_pred_humid

,Date,lokasi,rh2m...,prediction
0,2023-10-17 15:00:00,96001,80.3053,83.371353
1,2023-10-17 18:00:00,96001,77.4673,84.750320
2,2023-10-17 21:00:00,96001,81.3127,81.520134
3,2023-10-18 00:00:00,96001,80.4639,81.959892
4,2023-10-18 03:00:00,96001,75.7992,70.463509
...,...,...,...,...
4147,2023-10-20 00:00:00,99992,62.5816,80.262512
4148,2023-10-20 03:00:00,99992,40.4115,53.427265
4149,2023-10-20 06:00:00,99992,49.5108,48.920624
4150,2023-10-20 09:00:00,99992,57.6837,56.706219


In [19]:
# OUTPUT precipitation
df_pred_prec = generate_output_df('prec_nwp', ina_nwp_input, ina_nwp_input_filtered, prec_pred)
df_pred_prec

,Date,lokasi,prec_nwp,prediction
0,2023-10-17 15:00:00,96001,2.923480e-19,0.310729
1,2023-10-17 18:00:00,96001,1.034630e-19,0.282313
2,2023-10-17 21:00:00,96001,0.000000e+00,0.245382
3,2023-10-18 00:00:00,96001,0.000000e+00,0.234680
4,2023-10-18 03:00:00,96001,0.000000e+00,0.139990
...,...,...,...,...
4147,2023-10-20 00:00:00,99992,0.000000e+00,-0.176199
4148,2023-10-20 03:00:00,99992,0.000000e+00,-0.187036
4149,2023-10-20 06:00:00,99992,3.250240e-03,-0.137372
4150,2023-10-20 09:00:00,99992,1.239780e-05,-0.036114


In [20]:
data_table_lokasi_temp = create_data_table(df_wmoid, df_pred_temp, 'temp', merge_column='lokasi')
data_table_lokasi_temp

,lokasi,Nama UPT,day,Date,max_temp,mean_temp,min_temp
0,96001,Stasiun Meteorologi Maimun Saleh,17,2023-10-17,26.6,26.4,26.1
1,96001,Stasiun Meteorologi Maimun Saleh,18,2023-10-18,28.2,27.4,26.8
2,96001,Stasiun Meteorologi Maimun Saleh,19,2023-10-19,27.5,27.1,26.3
3,96001,Stasiun Meteorologi Maimun Saleh,20,2023-10-20,28.0,27.7,26.9
4,96009,Stasiun Meteorologi Malikussaleh,17,2023-10-17,25.5,25.4,25.2
...,...,...,...,...,...,...,...
671,97980,Stasiun Meteorologi Mopah,20,2023-10-20,31.6,29.7,26.3
672,99992,Balai Besar MKG Wilayah II,17,2023-10-17,28.6,26.3,25.0
673,99992,Balai Besar MKG Wilayah II,18,2023-10-18,33.5,29.2,25.0
674,99992,Balai Besar MKG Wilayah II,19,2023-10-19,33.5,28.8,24.4


In [21]:
data_table_lokasi_humid = create_data_table(df_wmoid, df_pred_humid, 'humidity', merge_column='lokasi')
data_table_lokasi_humid

,lokasi,Nama UPT,day,Date,max_humidity,mean_humidity,min_humidity
0,96001,Stasiun Meteorologi Maimun Saleh,17,2023-10-17,84.8,83.2,81.5
1,96001,Stasiun Meteorologi Maimun Saleh,18,2023-10-18,87.3,79.6,70.5
2,96001,Stasiun Meteorologi Maimun Saleh,19,2023-10-19,92.6,87.2,83.9
3,96001,Stasiun Meteorologi Maimun Saleh,20,2023-10-20,85.5,79.3,74.5
4,96009,Stasiun Meteorologi Malikussaleh,17,2023-10-17,98.7,95.4,93.2
...,...,...,...,...,...,...,...
671,97980,Stasiun Meteorologi Mopah,20,2023-10-20,88.5,77.2,68.9
672,99992,Balai Besar MKG Wilayah II,17,2023-10-17,82.2,77.8,70.1
673,99992,Balai Besar MKG Wilayah II,18,2023-10-18,87.3,69.9,50.5
674,99992,Balai Besar MKG Wilayah II,19,2023-10-19,84.8,66.9,39.6


In [22]:
data_table_lokasi_prec = create_data_table(df_wmoid, df_pred_prec, 'precipitation', merge_column='lokasi')
data_table_lokasi_prec

,lokasi,Nama UPT,day,Date,max_precipitation,mean_precipitation,min_precipitation
0,96001,Stasiun Meteorologi Maimun Saleh,17,2023-10-17,0.3,0.3,0.2
1,96001,Stasiun Meteorologi Maimun Saleh,18,2023-10-18,0.3,0.2,0.1
2,96001,Stasiun Meteorologi Maimun Saleh,19,2023-10-19,0.4,0.3,0.2
3,96001,Stasiun Meteorologi Maimun Saleh,20,2023-10-20,0.3,0.2,0.2
4,96009,Stasiun Meteorologi Malikussaleh,17,2023-10-17,0.4,0.3,0.3
...,...,...,...,...,...,...,...
671,97980,Stasiun Meteorologi Mopah,20,2023-10-20,0.1,0.1,-0.0
672,99992,Balai Besar MKG Wilayah II,17,2023-10-17,0.2,0.2,0.1
673,99992,Balai Besar MKG Wilayah II,18,2023-10-18,0.3,0.1,-0.1
674,99992,Balai Besar MKG Wilayah II,19,2023-10-19,0.2,0.1,-0.3


In [23]:
# Merge the dataframes
data_table_lokasi = data_table_lokasi_temp.merge(data_table_lokasi_humid.drop(columns=['Nama UPT', 'day']), on=['lokasi', 'Date'], how='left').merge(data_table_lokasi_prec.drop(columns=['Nama UPT', 'day']), on=['lokasi', 'Date'], how='left')
data_table_lokasi

,lokasi,Nama UPT,day,Date,max_temp,mean_temp,min_temp,max_humidity,mean_humidity,min_humidity,max_precipitation,mean_precipitation,min_precipitation
0,96001,Stasiun Meteorologi Maimun Saleh,17,2023-10-17,26.6,26.4,26.1,84.8,83.2,81.5,0.3,0.3,0.2
1,96001,Stasiun Meteorologi Maimun Saleh,18,2023-10-18,28.2,27.4,26.8,87.3,79.6,70.5,0.3,0.2,0.1
2,96001,Stasiun Meteorologi Maimun Saleh,19,2023-10-19,27.5,27.1,26.3,92.6,87.2,83.9,0.4,0.3,0.2
3,96001,Stasiun Meteorologi Maimun Saleh,20,2023-10-20,28.0,27.7,26.9,85.5,79.3,74.5,0.3,0.2,0.2
4,96009,Stasiun Meteorologi Malikussaleh,17,2023-10-17,25.5,25.4,25.2,98.7,95.4,93.2,0.4,0.3,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
671,97980,Stasiun Meteorologi Mopah,20,2023-10-20,31.6,29.7,26.3,88.5,77.2,68.9,0.1,0.1,-0.0
672,99992,Balai Besar MKG Wilayah II,17,2023-10-17,28.6,26.3,25.0,82.2,77.8,70.1,0.2,0.2,0.1
673,99992,Balai Besar MKG Wilayah II,18,2023-10-18,33.5,29.2,25.0,87.3,69.9,50.5,0.3,0.1,-0.1
674,99992,Balai Besar MKG Wilayah II,19,2023-10-19,33.5,28.8,24.4,84.8,66.9,39.6,0.2,0.1,-0.3


In [24]:
# Make geopandas geometry for coordinates
geometry = geopandas.points_from_xy(df_map.LON, df_map.LAT)
upt_gpd = geopandas.GeoDataFrame(df_map, geometry=geometry)
upt_gpd

,lokasi,Nama UPT,LON,LAT,geometry
0,96001,Stasiun Meteorologi Maimun Saleh,95.33785,5.87655,POINT (95.33785 5.87655)
24,96009,Stasiun Meteorologi Malikussaleh,96.94749,5.22869,POINT (96.94749 5.22869)
48,96011,Stasiun Meteorologi Sultan Iskandar Muda,95.41700,5.52244,POINT (95.41700 5.52244)
72,96015,Stasiun Meteorologi Cut Nyak Dhien Nagan Raya,96.24796,4.04928,POINT (96.24796 4.04928)
96,96017,Stasiun Klimatologi Aceh,95.46400,5.40400,POINT (95.46400 5.40400)
...,...,...,...,...,...
3936,97876,Stasiun Meteorologi Tanah Merah,140.31000,-6.10000,POINT (140.31000 -6.10000)
3960,97900,Stasiun Meteorologi Mathilda Batlayeri,131.30000,-7.98000,POINT (131.30000 -7.98000)
3984,97978,Stasiun Klimatologi Merauke,140.51700,-8.38700,POINT (140.51700 -8.38700)
4008,97980,Stasiun Meteorologi Mopah,140.41568,-8.52019,POINT (140.41568 -8.52019)


In [25]:
upt_gpd_merged = pd.merge(upt_gpd, data_table_lokasi[data_table_lokasi.drop(columns=['Nama UPT']).columns], on='lokasi')
upt_gpd_merged = upt_gpd_merged.reset_index(drop=True)
upt_gpd_merged

,lokasi,Nama UPT,LON,LAT,geometry,day,Date,max_temp,mean_temp,min_temp,max_humidity,mean_humidity,min_humidity,max_precipitation,mean_precipitation,min_precipitation
0,96001,Stasiun Meteorologi Maimun Saleh,95.33785,5.87655,POINT (95.33785 5.87655),17,2023-10-17,26.6,26.4,26.1,84.8,83.2,81.5,0.3,0.3,0.2
1,96001,Stasiun Meteorologi Maimun Saleh,95.33785,5.87655,POINT (95.33785 5.87655),18,2023-10-18,28.2,27.4,26.8,87.3,79.6,70.5,0.3,0.2,0.1
2,96001,Stasiun Meteorologi Maimun Saleh,95.33785,5.87655,POINT (95.33785 5.87655),19,2023-10-19,27.5,27.1,26.3,92.6,87.2,83.9,0.4,0.3,0.2
3,96001,Stasiun Meteorologi Maimun Saleh,95.33785,5.87655,POINT (95.33785 5.87655),20,2023-10-20,28.0,27.7,26.9,85.5,79.3,74.5,0.3,0.2,0.2
4,96009,Stasiun Meteorologi Malikussaleh,96.94749,5.22869,POINT (96.94749 5.22869),17,2023-10-17,25.5,25.4,25.2,98.7,95.4,93.2,0.4,0.3,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671,97980,Stasiun Meteorologi Mopah,140.41568,-8.52019,POINT (140.41568 -8.52019),20,2023-10-20,31.6,29.7,26.3,88.5,77.2,68.9,0.1,0.1,-0.0
672,99992,Balai Besar MKG Wilayah II,106.75636,-6.30253,POINT (106.75636 -6.30253),17,2023-10-17,28.6,26.3,25.0,82.2,77.8,70.1,0.2,0.2,0.1
673,99992,Balai Besar MKG Wilayah II,106.75636,-6.30253,POINT (106.75636 -6.30253),18,2023-10-18,33.5,29.2,25.0,87.3,69.9,50.5,0.3,0.1,-0.1
674,99992,Balai Besar MKG Wilayah II,106.75636,-6.30253,POINT (106.75636 -6.30253),19,2023-10-19,33.5,28.8,24.4,84.8,66.9,39.6,0.2,0.1,-0.3


In [28]:
geojson = json.loads(upt_gpd_merged.to_json())
geojson

{'type': 'FeatureCollection',
 'features': [{'id': '0',
   'type': 'Feature',
   'properties': {'lokasi': 96001,
    'Nama UPT': 'Stasiun Meteorologi Maimun Saleh',
    'LON': 95.33785,
    'LAT': 5.87655,
    'day': 17,
    'Date': '2023-10-17',
    'max_temp': 26.6,
    'mean_temp': 26.4,
    'min_temp': 26.1,
    'max_humidity': 84.8,
    'mean_humidity': 83.2,
    'min_humidity': 81.5,
    'max_precipitation': 0.3,
    'mean_precipitation': 0.3,
    'min_precipitation': 0.2},
   'geometry': {'type': 'Point', 'coordinates': [95.33785, 5.87655]}},
  {'id': '1',
   'type': 'Feature',
   'properties': {'lokasi': 96001,
    'Nama UPT': 'Stasiun Meteorologi Maimun Saleh',
    'LON': 95.33785,
    'LAT': 5.87655,
    'day': 18,
    'Date': '2023-10-18',
    'max_temp': 28.2,
    'mean_temp': 27.4,
    'min_temp': 26.8,
    'max_humidity': 87.3,
    'mean_humidity': 79.6,
    'min_humidity': 70.5,
    'max_precipitation': 0.3,
    'mean_precipitation': 0.2,
    'min_precipitation': 0.1},
 

In [30]:
geobuf = dlx.geojson_to_geobuf(geojson)
geobuf

'...'